# Smart Care Analytics

## Packages

In [ ]:
import pdfplumber
import pandas as pd
import re
import os
import openpyxl
import numpy as np
import csv


In [ ]:
# --- CONFIGURATION ---
# Remplacez par le nom exact de votre fichier PDF et le chemin de sortie désiré.
PDF_PATH = "fichier/SLP-CHF2012.pdf"
OUTPUT_EXCEL = "fichier/Rapport_Donnees_Extraites.xlsx"
OUTPUT_CSV = "fichier/Rapport_Donnees_Extraites.csv"

In [ ]:
def clean_text(text):
    """Nettoie le texte : enlève les retours à la ligne et les espaces doubles."""
    if text is None:
        return ""
    text = str(text).replace('\n', ' ').strip()
    text = re.sub('\s+', ' ', text)
    return text

def clean_numeric(val):
    if val is None or val == '':
        return None
    s = str(val).strip()
    s = re.sub(r'[^\d,.-]', '', s)
    s = s.replace(',', '.')
    try:
        return float(s) if '.' in s else int(s)
    except Exception:
        return val

def extraire_tableaux_pdf(pdf_path):
    """Extrait tous les tableaux d'un PDF et retourne une liste de DataFrame."""
    tables = []
    print(f\
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            page_tables = page.extract_tables()
            for t_idx, table in enumerate(page_tables):
                if not table:
                    continue
                df = pd.DataFrame(table)
                # Nettoyage texte brut
                df = df.applymap(clean_text)
                # Marquage de contexte
                df['source_page'] = i + 1
                df['table_id'] = f\

                tables.append(df)
    print(f\
    return tables

def detect_header_and_normalize(df):
    # Si la première ligne contient des valeurs non vides, l'utiliser comme header
    if df.shape[0] >= 1 and df.iloc[0].notna().sum() > 0:
        cols = [clean_text(c) if c else f\
 for i, c in enumerate(df.iloc[0])]
        df = df[1:].copy()
        df.columns = cols
    else:
        df.columns = [f\
 for i in range(df.shape[1])]
    return df

def nettoyer_et_formater_tables(dfs):
    """Prend une liste de DataFrame brutes, normalise les en-têtes, nettoie les cellules, convertit les nombres, et concatène en un dataset."""
    processed = []
    for df in dfs:
        if df.empty:
            continue
        df = detect_header_and_normalize(df)
        # Appliquer nettoyage cellule par cellule
        for col in df.columns:
            df[col] = df[col].apply(lambda x: clean_text(x))
        # Tenter de convertir les colonnes majoritairement numériques
        for col in df.columns:
            sample = df[col].dropna().head(20).astype(str)
            num_like = sample.str.match(r'^[\d\s\.,-]+$').sum()
            if len(sample) > 0 and num_like / len(sample) >= 0.6:
                df[col] = df[col].apply(clean_numeric)
        # Supprimer colonnes vides
        df = df.dropna(axis=1, how='all')
        # Ajouter au résultat
        processed.append(df)
    if not processed:
        return pd.DataFrame()
    final = pd.concat(processed, ignore_index=True, sort=False).drop_duplicates().reset_index(drop=True)
    # Suppression des lignes totalement vides
    final = final.dropna(how='all')
    return final

def sauvegarder_excel(df, output_xlsx, output_csv=None):
    if df is None or df.empty:
        print(\
)
        return
    os.makedirs(os.path.dirname(output_xlsx), exist_ok=True)
    df.to_excel(output_xlsx, index=False, engine='openpyxl')
    print(f\
    if output_csv:
        df.to_csv(output_csv, index=False, sep=';', encoding='utf-8-sig')
        print(f\

In [ ]:
# --- EXÉCUTION DU PIPELINE ---
if __name__ == "__main__":
    if os.path.exists(PDF_PATH):
        # 1. Extraction des tableaux
        dfs_bruts = extraire_tableaux_pdf(PDF_PATH)
        # 2. Nettoyage et formatage
        final_df = nettoyer_et_formater_tables(dfs_bruts)
        # 3. Sauvegarde
        sauvegarder_excel(final_df, OUTPUT_EXCEL, OUTPUT_CSV)
        # Aperçu rapide
        if not final_df.empty:
            print(final_df.head())
    else:
        print(f\

--- Démarrage de l'extraction sur : fichier/SLP-CHF2012.pdf ---
Traitement de la page 1...
Traitement de la page 2...
Traitement de la page 3...
Traitement de la page 4...
--- Sauvegarde de 14 tableaux uniques dans Rapport_Donnees_Extraites.xlsx ---
Terminé avec succès !


In [ ]:
# Cellule vide réservée pour tests interactifs